In [47]:
import pandas as pd
import requests
import json
import random
import openai

URL_BASE_API_SDW = "https://sdw-2023-prd.up.railway.app/"
openai.api_key = pd.read_csv("../credentials/credentials.csv")["gpt_key"][0]

In [48]:
# Função para buscar dados dos clientes/usuarios
def get_user(id):
    response = requests.get(f"{URL_BASE_API_SDW}users/{id}")

    return response.json() if response.status_code == 200 else None

In [49]:
# Busca os usuarios que existem dentro da API, para não trabalharmos com usuários não cadastrados
valid_users = requests.get(f'{URL_BASE_API_SDW}users')

# Seleciona aleatoriamente, 10 IDs para o projeto
ids = random.sample([x["id"] for x in valid_users.json()],10)

In [50]:
# Busca dados de cada usuário
users =  [get_user(x) for x in ids]

In [51]:
# Esse dicionário deverá coletar os dados extraídos 
users_data = {  "id":     [x["id"] for x in users],
                "name":   [x["name"] for x in users]}

# Os dados serão adicionados aos já inseridos no users_data
for k in users:
    for i, j in k.items():
        if i in ["id","name"]:
            continue

        elif i in ["features","news"]:
            continue
        
        else:
            for l, m in j.items():
                column_name = f"{i}_{l}"

                if not(column_name in users_data.keys()):
                    users_data[column_name] = []
                
                users_data[column_name].append(m)

# Transforma users_data em dataframe para manuseio futuro
users_data = pd.DataFrame(users_data)

In [67]:
def generate_marketing(name):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[
            {"role": "system", 
            "content": "Você é um gerente de marketing da empresa de celular Samsung"},
            {"role": "user",
             "content": f"Crie uma mensagem de marketing personalizada para {name} (resposta limitada a 50 tokens)"}
        ]
    )
    return completion.choices[0].message["content"].strip('"')

In [68]:
base_marketing = {
    "name":[],
    "menssage": []
}

for nome in random.sample(list(users_data["name"]),3):
    base_marketing["name"].append(nome)
    base_marketing["menssage"].append(generate_marketing(nome))

base_marketing = pd.DataFrame(base_marketing)

In [ ]:
#TODO Falta implementar sistema de carregamento do conteúdo, já enriquecido.